# Post-Hoc Rule Optimization: Merging & Pruning

**Objective:**
After extracting the initial set of decision rules for **CCF detection**, our goal is to simplify the model without significantly sacrificing performance (specifically **Recall**). We apply a two-stage optimization process:

1.  **Rule Merging (Pairwise Analysis):**
    We iteratively test merging every possible pair of rules (by identifying their common logical ancestors). This helps us find broader, more generalized rules that can replace specific variations.

2.  **Rule Pruning (Leave-One-Out Test):**
    We perform a sensitivity analysis to identify **redundant rules**. By removing one rule at a time and observing the drop in Recall, we can distinguish between "critical rules" (must-haves) and "optional rules" (safe to remove).

### Environment & Data Loading

In [33]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

# ===================== Configuration =====================
DATA_PATH  = "XAI_Drilling_Dataset.csv"
TARGET     = "CCF"
SPLIT_SEED = 259
TEST_SIZE  = 0.20
# =========================================================

# Column Mapping
CANONICAL_MAP = {
    "Cutting_speed":   ["Cutting_speed", "Cutting speed vc [m/min]"],
    "Spindle_speed":   ["Spindle_speed", "Spindle speed n [1/min]"],
    "Feed_f":          ["Feed_f", "Feed f [mm/rev]"],
    "Feed_rate":       ["Feed_rate", "Feed rate vf [mm/min]"],
    "Power_Pc":        ["Power_Pc", "Power Pc [kW]"],
    "Cooling":         ["Cooling", "Cooling [%]"],
    "Material":        ["Material"],
    "Drill_Bit_Type":  ["Drill_Bit_Type", "Drill Bit Type"],
    "Process_Time":    ["Process_Time", "Process Time [sec]"],
    "BEF": ["BEF"], "CCF": ["CCF"], "FWF": ["FWF"], "WDF": ["WDF"]
}

def _safe_get(df, variants):
    for c in variants:
        if c in df.columns: return df[c]
    raise KeyError(f"Missing columns: {variants}")

def load_dataset(path):
    df0 = pd.read_csv(path)
    df = pd.DataFrame()
    for canon, variants in CANONICAL_MAP.items():
        try: df[canon] = _safe_get(df0, variants)
        except KeyError: pass
    return df

def prepare_features(df, target):
    base_cols = ["Cutting_speed", "Spindle_speed", "Feed_f", "Feed_rate", "Power_Pc", "Cooling", "Process_Time", "Material", "Drill_Bit_Type"]
    use_cols = [c for c in base_cols if c in df.columns]
    X = df[use_cols].copy()
    y = df[target].astype(int).copy()

    # Numeric Cleaning
    for c in ["Cutting_speed", "Spindle_speed", "Feed_f", "Feed_rate", "Power_Pc", "Cooling", "Process_Time"]:
        if c in X.columns:
            X[c] = pd.to_numeric(X[c], errors="coerce").fillna(X[c].median())

    # Categorical Encoding
    for c in ["Material", "Drill_Bit_Type"]:
        if c in X.columns:
            X[c + "_enc"] = X[c].astype("category").cat.codes

    return X, y

### Logic Definition

To perform precise merging operations, we decompose the rule logic into **"Atoms"**.

* **Atoms:** The fundamental building blocks of our rules.
* **Rule Construction:** Each rule is defined as a specific combination of these atoms.
* **Input Configuration:**
    We define the **baseline set of rules** to be analyzed in this experiment.
    *(Note: The number of rules and their specific conditions below can be updated to reflect any rule set—whether extracted directly from a tree or pre-simplified in a previous step.)*

In [ ]:
def get_rule_structs(X):
    """
    Defines the exact atomic conditions and rule structures.
    This manual definition allows for precise control over rule boundaries.
    """
    # Feature shortcuts
    C  = X["Cooling"]
    FR = X["Feed_rate"]
    P  = X["Power_Pc"]
    DB = X["Drill_Bit_Type_enc"]
    CS = X["Cutting_speed"]
    MT = X["Material_enc"]
    Ff = X["Feed_f"]
    PT = X["Process_Time"]

    # 1. Define Atoms (The Building Blocks)
    atom_masks = {
        "C<=37.50":   (C <= 37.50), "C>37.50":    (C > 37.50), "C>62.50":    (C > 62.50),
        "FR>62.50":   (FR > 62.50), "FR>70.50":   (FR > 70.50),
        "FR>124.50":  (FR > 124.50), "FR>125.50":  (FR > 125.50), "FR<=124.50": (FR <= 124.50),
        "FR>205.50":  (FR > 205.50), "FR>206.50":  (FR > 206.50), "FR<=205.50": (FR <= 205.50),
        "P>63.52":    (P > 63.52), "P>64.54":    (P > 64.54), "P<=64.54":   (P <= 64.54),
        "DB<=1.50":   (DB <= 1.50), "DB>1.50":    (DB > 1.50), "DB<=0.50":   (DB <= 0.50), "DB>0.50":    (DB > 0.50),
        "CS>16.80":   (CS > 16.80), "CS<=16.80":  (CS <= 16.80), "CS<=17.03":  (CS <= 17.03),
        "MT>1.00":    (MT > 1.00),
        "Ff>0.20":    (Ff > 0.20), "Ff<=0.28":   (Ff <= 0.28),
        "PT>35.10":   (PT > 35.10), "PT>35.34":   (PT > 35.34),
    }

    # 2. Define The 8 Rules (Composed of Atoms)
    # Rule 1 is a special merged group (1+5+8 from original tree)
    rule_atoms = {
        1: ["C<=37.50", "FR>124.50", "P>63.52"],
        2: ["C>37.50", "FR>205.50", "FR>206.50", "DB<=1.50"],
        3: ["C<=37.50", "FR<=124.50", "DB>1.50"],
        4: ["C<=37.50", "FR<=124.50", "DB<=1.50", "FR>62.50", "FR>70.50", "CS>16.80", "DB<=0.50", "MT>1.00"],
        5: ["C<=37.50", "FR<=124.50", "DB<=1.50", "FR>62.50", "FR>70.50", "CS<=16.80", "DB>0.50"],
        6: ["C<=37.50", "FR<=124.50", "DB<=1.50", "FR>62.50", "FR>70.50", "CS>16.80", "DB>0.50", "Ff>0.20"],
        7: ["C>37.50", "FR<=205.50", "Ff<=0.28", "PT>35.10", "DB<=1.50", "DB<=0.50", "C>62.50"],
        8: ["C>37.50", "FR<=205.50", "Ff<=0.28", "PT>35.10", "DB>1.50", "Ff>0.20"],
    }

    # Generate Masks
    new_masks = {}
    for idx, atoms in rule_atoms.items():
        m = np.ones(len(X), dtype=bool)
        for a in atoms:
            m = m & atom_masks[a]
        new_masks[idx] = m
        
    orig_masks = {}
    
    return new_masks, atom_masks, rule_atoms

### Evaluation Functions

In [35]:
def evaluate_config(name, y_te, mask_bool):
    y_pred = mask_bool.astype(int)
    
    cm = confusion_matrix(y_te, y_pred)
    p = precision_score(y_te, y_pred, zero_division=0)
    r = recall_score(y_te, y_pred, zero_division=0)
    f = f1_score(y_te, y_pred, zero_division=0)
    
    missed = (y_te.values == 1) & (y_pred == 0)
    n_missed = missed.sum()
    
    return {
        "Configuration": name,
        "Precision": p,
        "Recall": r,
        "F1": f,
        "Missed_Failures": n_missed
    }

## Merging

In [ ]:
# 1. Load Data
df = load_dataset(DATA_PATH)
X, y = prepare_features(df, TARGET)
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=TEST_SIZE, stratify=y, random_state=SPLIT_SEED)

print(f"Test Set Size: {len(X_te)} | Failures in Test: {y_te.sum()}")

# 2. Get Rules
new_masks, atom_masks, rule_atoms = get_rule_structs(X_te)
results = []

# --- Baseline: All 8 Rules Active ---
mask_all8 = np.zeros(len(X_te), dtype=bool)
for idx in range(1, 9):
    mask_all8 |= new_masks[idx]
results.append(evaluate_config("Baseline (Rules 1-8)", y_te, mask_all8))

# --- Pairwise Merge Loop ---
print("Running pairwise merge analysis...")
for i in range(1, 9):
    for j in range(i+1, 9):
        shared = set(rule_atoms[i]) & set(rule_atoms[j])
        
        if len(shared) == 0:
            parent_mask = np.ones(len(X_te), dtype=bool)
        else:
            parent_mask = np.ones(len(X_te), dtype=bool)
            for a in shared:
                parent_mask &= atom_masks[a]
        
        merged_mask = parent_mask.copy()
        for k in range(1, 9):
            if k != i and k != j:
                merged_mask |= new_masks[k]
        
        results.append(evaluate_config(f"Merge R{i} + R{j}", y_te, merged_mask))

# 3. Summary Table
df_res = pd.DataFrame(results)
df_res = df_res.sort_values(by=["F1", "Recall"], ascending=False).reset_index(drop=True)

display(df_res.head(10))

best = df_res.iloc[0]
print(f"\nBest Strategy: {best['Configuration']}")
print(f"Recall: {best['Recall']:.3f} (Missed: {best['Missed_Failures']})")

Test Set Size: 4000 | Failures in Test: 69
Running pairwise merge analysis...


,Configuration,Precision,Recall,F1,Missed_Failures
0,Baseline (Rules 1-8),0.958333,1.0,0.978723,0
1,Merge R7 + R8,0.138000,1.0,0.242531,0
2,Merge R5 + R6,0.130930,1.0,0.231544,0
3,Merge R4 + R6,0.087011,1.0,0.160093,0
4,Merge R4 + R5,0.084044,1.0,0.155056,0
5,Merge R3 + R4,0.083942,1.0,0.154882,0
6,Merge R3 + R5,0.083942,1.0,0.154882,0
7,Merge R3 + R6,0.083942,1.0,0.154882,0
8,Merge R1 + R3,0.083738,1.0,0.154535,0
9,Merge R1 + R4,0.083738,1.0,0.154535,0



Best Strategy: Baseline (Rules 1-8)
Recall: 1.000 (Missed: 0)


## Elimination

In [41]:

# Reuse the rules defined in the previous step
# Assuming 'new_masks' and 'y_te' are available from the cells above

results_drop = []
target_rules = list(range(1, 9)) 

print(f"Running Leave-One-Out Test on 8 Rules...")

# Baseline
mask_all8 = np.zeros(len(X_te), dtype=bool)
for r in target_rules:
    mask_all8 |= new_masks[r]
results_drop.append(evaluate_config("Baseline (Keep All 8)", y_te, mask_all8))

# Drop-One Loop
for drop_id in target_rules:
    current_mask = np.zeros(len(X_te), dtype=bool)
    kept_list = []
    
    for r in target_rules:
        if r != drop_id:
            current_mask |= new_masks[r]
            kept_list.append(r)
            
    # Record result
    name = f"Drop Rule {drop_id}"
    res = evaluate_config(name, y_te, current_mask)
    res["Kept_Rules"] = str(kept_list)
    results_drop.append(res)

# Create DataFrame
df_drop = pd.DataFrame(results_drop)

#Sort by Recall

df_drop = df_drop.sort_values(by=["Recall", "F1"], ascending=[True, False]).reset_index(drop=True)

print("\n=== Elimination Results (Sorted by Recall Impact) ===")
print("Lower Recall = This rule was CRITICAL (Dropping it hurt performance)")
display(df_drop[["Configuration", "Recall", "Precision", "F1", "Missed_Failures"]])

Running Leave-One-Out Test on 8 Rules...

=== Elimination Results (Sorted by Recall Impact) ===
Lower Recall = This rule was CRITICAL (Dropping it hurt performance)


,Configuration,Recall,Precision,F1,Missed_Failures
0,Drop Rule 1,0.333333,0.958333,0.494624,46
1,Drop Rule 2,0.855072,0.951613,0.900763,10
2,Drop Rule 3,0.913043,0.954545,0.933333,6
3,Drop Rule 4,0.956522,0.956522,0.956522,3
4,Drop Rule 7,0.985507,0.971429,0.978417,1
5,Drop Rule 5,0.985507,0.957746,0.971429,1
6,Drop Rule 6,0.985507,0.957746,0.971429,1
7,Drop Rule 8,0.985507,0.957746,0.971429,1
8,Baseline (Keep All 8),1.000000,0.958333,0.978723,0
